## Deep Biomarkers Of Human Aging

#### How old by basic blood test

Age predictor needs all blood marker values to make a prediction. If you are missing values and filling them in, this might have a dramatic impact on the results. For example, male samples can be classified as females and vice versa. Neural network looks on the net totality of blood parameters, but not individual markers. 

In [1]:
import math
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd

# In[3]:

MODEL_PATH = Path.cwd()/'Models'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.df = df
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = self.df.drop(labels='AGE', axis=1).iloc[idx]
        x = torch.from_numpy(x.values)
        #returning y as a scalar might be a problem
        y = self.df.iloc[idx].AGE
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y


# In[5]:


class NN(nn.Module):
    def __init__(self, layers=[1000,500,250], ps=0.35, in_features=20, y_range=(20, 90)):
        super(NN, self).__init__()
        self.y_range = y_range
        self.layers = layers
        self.ps = ps
        layers = [in_features] + layers
        layers = list(zip(layers, layers[1:]))
        
        l = []
        for layer in layers:
            l.append(nn.Linear(*layer))
            #TODO: play with negative slope koef. of LeakyReLU
            l.append(nn.LeakyReLU())
            l.append(nn.Dropout(ps))
        l.append(nn.Linear(layers[-1][1], 1))

        self.arch = nn.Sequential(*l)
        
    def forward(self, x):
        x = self.arch(x)
        x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x
    
    def __repr__(self):
        return "Linear -> LeakyReLU -> Dropout\nlayers: {}\nps: {}\n".format(self.layers, self.ps)

In [3]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
from ipywidgets import GridspecLayout, Button, Layout, Dropdown, BoundedFloatText

In [4]:
import torch
from pathlib import Path

In [5]:
#MALe = 1 
#FEmale = 0
#Like www.aging.ai Western Europe except for:
#Albumine, Creatine urine isntead of blood

#Albuming check but probably OK

#Creatine just take the mean
#Protein total/10 


MALE40 = [
    49.21,
    5.28,
    7.3,
    7.04,
    7,
    143,
    120,
    16.5,
    17.2,
    1.35,
    1.6,
    4.83,
    2.48,
    4.7,
    46.7,
    35.4,
    86.5,
    177,
    5.41
]

In [6]:
MALE69 = [
    37,
    5.16,
    3.89,
    4.73,
    5.9,
    140,
    88.4,
    14.4,
    18.81,
    1.11,
    1.05,
    3.13,
    2.05,
    4.4,
    44.2,
    32.6,
    91,
    188,
    4.86
]

In [7]:
MALE29 = [
    47.64,
    5.06,
    6,
    5.35,
    8.2,
    139,
    72.7,
    13.6,
    10.9,
    0.74,
    1.38,
    3.63,
    2.62,
    4.7,
    40.8,
    33.3,
    73.2,
    206,
    5.57,
]

In [8]:
a="""Albumin**                        [                    ] 35 - 52 g/l
Glucose**                        [                    ] 3.9 - 5.8 mmole/l
Urea**(BUN)                      [                    ] 2.5 - 6.4 mmole/l
Cholesterol**                    [                    ] 3.37 - 5.96 mmole/l
Protein total**                  [                    ] 64 - 83 g/l
Sodium**                         [                    ] 136 - 146 mmole/l
Creatinine**                     [                    ] 53 - 97 mmole/l
Hemoglobin**                     [                    ] 11.7 - 15.5 g/dl
Bilirubin total                  [                    ] 1.7 - 21 mcmole/l
Triglycerides                    [                    ] 0.68 - 6 mmole/l
HDL Cholesterol                  [                    ] < 3.3 mmole/l
LDL cholesterol (by Friedewald)  [                    ] 1.81- 4.04 mmole/l
Calcium                          [                    ] 2.15 - 2.65 mmole/l
Potassium                        [                    ] 3.4 - 5.1 mmole/l
Hematocrit                       [                    ] 37 - 50 %
MCHC                             [                    ] 31.5 - 35.7 g/dL
MCV                              [                    ] 82 - 95 fl
Platelets                        [                    ] 150 - 450 10^3 /mcl
Erythrocytes (RBC)               [                    ] 3.5 - 5.5 10^6 /mcl"""

In [9]:
lines = a.splitlines()
lines = [l.split('[') for l in lines]
lines = [(b.strip(), m.strip()) for b, m in lines]

In [10]:
biomarkers = [l[0] for l in lines]

In [11]:
measures = [l[1][2:] for l in lines]
measures[4] = '6.4 - 8.3 g/dl'

In [12]:
biomarkers = [(i+1, biomarkers[i], measures[i]) for i in range(len(biomarkers))]

In [13]:
#('hand written code', Path.cwd()/'Models'/'adm_state_dict', (Path.cwd()/'Models'/'adm_state_dict').exists())

In [14]:
def predict(biomarkers):
    path = Path.cwd()/'Models'/'adm_state_dict'
    lm = NN()
    lm.load_state_dict(torch.load(path))
    lm.eval()
    biomarkers = torch.Tensor(biomarkers)
    return(lm(biomarkers))

In [15]:
from ipywidgets import *
x = widgets.Dropdown(options=['a', 'b'], description="Load an example")
y = widgets.Dropdown(options=["Male", "Female"], description="Sex:")

def change_x(*args):
    y.index = None
    y.value = "Male"
    
    for i in range(len(MALE29)+1):
        grid[i+1, 2].value = MALE29[i]
        
x.observe(change_x, 'value')
HBox([x,y])

In [16]:
def submit_clicked(b):
    markers = [grid[i+1, 2].value for i in range(len(biomarkers))]
    markers.insert(0, int(y.value=='Male'))
    with output:
        print(f"You are {round(predict(markers).data.item())} years old.")
        

In [17]:
from ipywidgets import GridspecLayout

grid = widgets.GridspecLayout(len(biomarkers)+2, 5, height='800px', width='auto')
h = '28px'
for x in biomarkers:
        grid[x[0], 1] = Button(description=x[1], tooltip=x[1], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        grid[x[0], 3] = Button(description=x[2], tooltip=x[2], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        
        grid[x[0], 2] = BoundedFloatText(
        min=0,
        max=1000,
        layout=Layout(width='auto', height=h),
        value=None,
        step=0.001,
        #description=' ',
        disabled=False
    )
        
grid[0,1] = Button(description="Blood Marker*", tooltip="Units and Sample Metric***", button_style='primary', disabled=True, layout=Layout(height='auto', width='auto'))
grid[0,2] = Button(description="Your Value", tooltip="Your Value", button_style="primary", disabled=True, layout=Layout(height='auto', width='auto'))
grid[0,3] = Button(description="Units and Sample Metric***", tooltip="Units and Sample Metric***", button_style="primary", disabled=True, layout=Layout(height='auto', width='auto'))
        
grid

GridspecLayout(children=(Button(button_style='info', description='Albumin**', disabled=True, layout=Layout(gri…

In [18]:
button = widgets.Button(description="Submit")
button.on_click(submit_clicked)
output = widgets.Output()
display(button, output)

Button(description='Submit', style=ButtonStyle())

Output()

\* This should be in your clinical biochemistry blood test results  
\** Required parameter for minimal prediction accuracy of 70% within 10 year frame  
\*** We can not show you reference values before knowing your age apriori, so this is only a reference metric  

Click on a marker to know more about what it is. Knowing your blood biochemistry markers is the new ABCs. Everyone should know it.   

This is a deep-learned predictor of your age made with a deep neural network trained on hundreds of thousands anonymized human blood tests. Enter your data below and Aging.ai will guess your age.

Please use the following citation when referencing Aging.AI 3.0: Polina Mamoshina, Kirill Kochetov, Evgeny Putin, Franco Cortese, Alexander Aliper, Won-Suk Lee, Sung-Min Ahn, Lee Uhn, Neil Skjodt, Olga Kovalchuk, Morten Scheibye-Knudsen, Alex Zhavoronkov; Population specific biomarkers of human aging: a big data study using South Korean, Canadian and Eastern European patient populations, The Journals of Gerontology: Series A, gly005, https://doi.org/10.1093/gerona/gly005

Made by Dmytro Mishchenko as a project for a Machine Learning Club at San Jose State University.    
https://mlatsjsu.club  
This project is open-source and is covered by the MIT License, for more details follow the GitHub link below.  
https://github.com/mdmittriy/AgingClock  